<h1>Book Recommendation Engine using KNN ML</h1>

In [4]:
# import libraries (you may add additional imports but you may not have to)

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import hstack
import matplotlib.pyplot as plt

In [1]:
# get data files

!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-03-25 23:11:09--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24,88M  37,3MB/s    in 0,7s    

2024-03-25 23:11:10 (37,3 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [69]:
# import csv data into dataframes

df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [70]:
# show books dataframe

df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [71]:
# show ratings dataframe

df_ratings.sample(5)

,user,isbn,rating
192930,42914,0808740415,0.0
313982,75709,0671880527,5.0
61580,12538,0449203794,0.0
997810,239554,0316089699,0.0
534193,128988,0688128165,0.0


In [72]:
# add your code here - consider creating a new cell for each section of code

# 1) Clean the data

# first, for the books
nan_count_books = df_books.isna().sum()

print(nan_count_books)

isbn      0
title     0
author    2
dtype: int64


In [73]:
# see those books with no author

filt = df_books['author'].isna()
df_books[filt]

,isbn,title,author
118038,0751352497,A+ Quiz Masters:01 Earth,NaN
187700,9627982032,The Credit Suisse Guide to Managing Your Perso...,NaN


In [74]:
# remove them

df_books.dropna(inplace = True)

nan_count_books = df_books.isna().sum()

print(nan_count_books)

isbn      0
title     0
author    0
dtype: int64


In [75]:
# second, for the ratings (no NaN values here)

nan_count_ratings = df_ratings.isna().sum()

print(nan_count_ratings)

user      0
isbn      0
rating    0
dtype: int64


In [76]:
# remove from the dataset users with less than 200 ratings and books with less than 100 ratings

count_user = (df_ratings['user'].value_counts() > 200)
count_title = (df_ratings['isbn'].value_counts() > 100)

filt_user = df_ratings['user'].isin(count_user[count_user].index)
filt_title = df_ratings['isbn'].isin(count_title[count_title].index)
df_ratings = df_ratings[filt_user]
df_ratings = df_ratings[filt_title]
df_ratings

/tmp/ipykernel_17458/1428827234.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_ratings = df_ratings[filt_title]


,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [77]:
# merge ratings with books
new_df = pd.merge(right = df_ratings, left = df_books, on = "isbn")

new_df.drop_duplicates(["title", "user"], inplace = True)

piv_df = new_df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
piv_df

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
# make the matrix for training
matrix = piv_df.values
matrix.shape

(664, 882)

In [84]:
# train the model
knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
knn.fit(matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [87]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  X = piv_df.loc[book].array.reshape(1, -1)
  d, i = knn.kneighbors(X, n_neighbors = 5)
  rated_books = []
  for distance, indice in zip(d[0], i[0]):
    if distance != 0:
      rated_book = piv_df.index[indice]
      rated_books.append([rated_book, distance])
  recommended_books = [book, rated_books[::-1]]
  return recommended_books

In [88]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Weight of Water', 0.7708583572697412], ['The Surgeon', 0.7699410973804288], ['I Know This Much Is True', 0.7677075092617776], ['The Lovely Bones: A Novel', 0.723018438082393], ["Where the Heart Is (Oprah's Book Club (Paperback))", 2.220446049250313e-16]]]
You passed the challenge! 🎉🎉🎉🎉🎉
